In [1]:
import random
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# To print multiple outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
# Set it to None to display all columns in the dataframe
pd.set_option('display.max_columns', None)


In [3]:
# data = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/fake_job_postings.csv")
data = pd.read_csv("./newFile.csv")
data

,title,salary_range,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,Country,State,City,has_company_profile
0,Marketing Intern,NaN,"Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0,US,NY,New York,1
1,Customer Service - Cloud Video Production,NaN,Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0,NZ,,Auckland,1
2,Commissioning Machinery Assistant (CMA),NaN,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,Other,0,US,IA,Wever,1
3,Account Executive - Washington DC,NaN,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0,US,DC,Washington,1
4,Bill Review Manager,NaN,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0,US,FL,Fort Worth,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,Account Director - Distribution,NaN,Just in case this is the first time you’ve vis...,To ace this role you:Will eat comprehensive St...,What can you expect from us?We have an open cu...,0,1,1,Full-time,Mid-Senior level,NaN,Computer Software,Sales,0,CA,ON,Toronto,1
17876,Payroll Accountant,NaN,The Payroll Accountant will focus primarily on...,- B.A. or B.S. in Accounting- Desire to have f...,Health &amp; WellnessMedical planPrescription ...,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Internet,Accounting/Auditing,0,US,PA,Philadelphia,1
17877,Project Cost Control Staff Engineer - Cost Con...,NaN,Experienced Project Cost Control Staff Enginee...,At least 12 years professional experience.Abil...,NaN,0,0,0,Full-time,NaN,NaN,NaN,engineer,0,US,TX,Houston,1
17878,Graphic Designer,NaN,Nemsia Studios is looking for an experienced v...,1. Must be fluent in the latest versions of Co...,Competitive salary (compensation will be based...,0,0,1,Contract,Not Applicable,Professional,Graphic Design,Design,0,NG,LA,Lagos,0


In [4]:
data.describe()

,telecommuting,has_company_logo,has_questions,fraudulent,has_company_profile
count,17880.000000,17880.000000,17880.000000,17880.000000,17880.000000
mean,0.042897,0.795302,0.491723,0.048434,0.814989
std,0.202631,0.403492,0.499945,0.214688,0.388317
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,0.000000,1.000000
50%,0.000000,1.000000,0.000000,0.000000,1.000000
75%,0.000000,1.000000,1.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
data.isnull ().sum ()

title                      0
salary_range           15012
description                1
requirements            2695
benefits                7210
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         3471
required_experience     7050
required_education      8105
industry                4903
function                   0
fraudulent                 0
Country                  346
State                    440
City                     440
has_company_profile        0
dtype: int64

In [7]:
# df = data.drop('job_id', axis=1)
df = data.drop('salary_range', axis=1)
df.tail()

,title,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,Country,State,City,has_company_profile
17875,Account Director - Distribution,Just in case this is the first time you’ve vis...,To ace this role you:Will eat comprehensive St...,What can you expect from us?We have an open cu...,0,1,1,Full-time,Mid-Senior level,NaN,Computer Software,Sales,0,CA,ON,Toronto,1
17876,Payroll Accountant,The Payroll Accountant will focus primarily on...,- B.A. or B.S. in Accounting- Desire to have f...,Health &amp; WellnessMedical planPrescription ...,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Internet,Accounting/Auditing,0,US,PA,Philadelphia,1
17877,Project Cost Control Staff Engineer - Cost Con...,Experienced Project Cost Control Staff Enginee...,At least 12 years professional experience.Abil...,NaN,0,0,0,Full-time,NaN,NaN,NaN,engineer,0,US,TX,Houston,1
17878,Graphic Designer,Nemsia Studios is looking for an experienced v...,1. Must be fluent in the latest versions of Co...,Competitive salary (compensation will be based...,0,0,1,Contract,Not Applicable,Professional,Graphic Design,Design,0,NG,LA,Lagos,0
17879,Web Application Developers,Who are we?Vend is an award winning web based ...,We want to hear from you if:You have an in-dep...,NaN,0,1,1,Full-time,Mid-Senior level,NaN,Computer Software,Engineering,0,NZ,N,Wellington,1


In [11]:
# df['department'] = df['department'].fillna('Other')
# df['location'] = df['location'].fillna('Unknown')
# df['company_profile'] = df['company_profile'].fillna('No Company Profile')
df['description'] = df['description'].fillna('No Description')
df['requirements'] = df['requirements'].fillna('Unknown Requirements')
df['benefits'] = df['benefits'].fillna('No Listed Benefits')
df['employment_type'] = df['employment_type'].fillna('Unknown')
df['required_experience'] = df['required_experience'].fillna('Unknown')
df['required_education'] = df['required_education'].fillna('Unknown')
df['industry'] = df['industry'].fillna('Unknown')
df['function'] = df['function'].fillna('Unknown')
df['Country'] = df['Country'].fillna('Unknown')
df['State'] = df['State'].fillna('Unknown')
df['City'] = df['City'].fillna('Unknown')

In [12]:
df.isna().sum()

title                  0
description            0
requirements           0
benefits               0
telecommuting          0
has_company_logo       0
has_questions          0
employment_type        0
required_experience    0
required_education     0
industry               0
function               0
fraudulent             0
Country                0
State                  0
City                   0
has_company_profile    0
dtype: int64

In [25]:
import regex as re
import regex as re

df['description'] = df['description'].str.lower()
df['description'] = df['description'].apply(lambda x: re.sub(r'[^\w\d\s\']+', '', x))


In [13]:
#Tokenizing description series
import nltk 
from nltk.tokenize import word_tokenize

df['description_token'] = df['description'].apply(word_tokenize)
df.head(1)

,title,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,Country,State,City,has_company_profile,description_token
0,Marketing Intern,"Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,No Listed Benefits,0,1,0,Other,Internship,Unknown,Unknown,Marketing,0,US,NY,New York,1,"[Food52, ,, a, fast-growing, ,, James, Beard, ..."


In [14]:
#remove stopwords from description
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
df['description_clean'] =  df['description_token'].apply(lambda x: [word for word in x if word not in stop_words])
df.head(1)

,title,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,Country,State,City,has_company_profile,description_token,description_clean
0,Marketing Intern,"Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,No Listed Benefits,0,1,0,Other,Internship,Unknown,Unknown,Marketing,0,US,NY,New York,1,"[Food52, ,, a, fast-growing, ,, James, Beard, ...","[Food52, ,, fast-growing, ,, James, Beard, Awa..."


In [15]:
#join string back for analysis

def combine_words(words_list):
    # Combine the words into a single string with spaces
    combined_string = " ".join(words_list)
    
    return combined_string


df["Total"] = df["description_clean"].apply(combine_words)

# Print the DataFrame
df.head()

,title,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,Country,State,City,has_company_profile,description_token,description_clean,Total
0,Marketing Intern,"Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,No Listed Benefits,0,1,0,Other,Internship,Unknown,Unknown,Marketing,0,US,NY,New York,1,"[Food52, ,, a, fast-growing, ,, James, Beard, ...","[Food52, ,, fast-growing, ,, James, Beard, Awa...","Food52 , fast-growing , James Beard Award-winn..."
1,Customer Service - Cloud Video Production,Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,Unknown,Marketing and Advertising,Customer Service,0,NZ,,Auckland,1,"[Organised, -, Focused, -, Vibrant, -, Awesome...","[Organised, -, Focused, -, Vibrant, -, Awesome...",Organised - Focused - Vibrant - Awesome ! Do p...
2,Commissioning Machinery Assistant (CMA),"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,No Listed Benefits,0,1,0,Unknown,Unknown,Unknown,Unknown,Other,0,US,IA,Wever,1,"[Our, client, ,, located, in, Houston, ,, is, ...","[Our, client, ,, located, Houston, ,, actively...","Our client , located Houston , actively seekin..."
3,Account Executive - Washington DC,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0,US,DC,Washington,1,"[THE, COMPANY, :, ESRI, –, Environmental, Syst...","[THE, COMPANY, :, ESRI, –, Environmental, Syst...",THE COMPANY : ESRI – Environmental Systems Res...
4,Bill Review Manager,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0,US,FL,Fort Worth,1,"[JOB, TITLE, :, Itemization, Review, ManagerLO...","[JOB, TITLE, :, Itemization, Review, ManagerLO...",JOB TITLE : Itemization Review ManagerLOCATION...


In [16]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

transformer = TfidfTransformer(smooth_idf = False)
count_vectorizer = CountVectorizer(ngram_range = (1,2))

In [17]:
train_counts = count_vectorizer.fit_transform(df['Total'].values)

train_tfidf = transformer.fit_transform(train_counts)
train_counts


<17880x708885 sparse matrix of type '<class 'numpy.int64'>'
	with 3647697 stored elements in Compressed Sparse Row format>

In [18]:
X = train_tfidf
y = df['fraudulent']

SMOTE
Way 1:

In [19]:
from collections import Counter
Counter(y)

Counter({0: 17014, 1: 866})

In [24]:
from imblearn.over_sampling import SMOTE
smo = SMOTE(random_state=42)
X_smo,y_smo = smo.fit_resample(X,y) 

AttributeError: 'NoneType' object has no attribute 'split'

In [40]:
Counter(y_smo)

Counter({0: 17014, 1: 17014})

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_smo,y_smo, test_size = 0.2, random_state = 3)

Build the Model

In [43]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

Adab = AdaBoostClassifier(DecisionTreeClassifier(max_depth = 3), n_estimators = 5)
Adab.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3),
                   n_estimators=5)

In [44]:
print("score for training dataset:",Adab.score(X_train, y_train))
print("score for test dataset:",Adab.score(X_test, y_test))
print("predict:",Adab.predict(X_test))

score for training dataset: 0.8310924987142753
score for test dataset: 0.8320599471054951
predict: [1 1 1 ... 1 1 0]


Way 2: Not works


In [ ]:
import random
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [ ]:
class Smote:

    def __init__(self, sampling_rate=5, k=5):
        self.sampling_rate = sampling_rate
        self.k = k
        self.newindex = 0

    def fit(self, X, y=None):
        if y is not None:
            negative_X = X[y==0]
            X = X[y==1]
            
        n_samples, n_features = X.shape
        
        self.synthetic = np.zeros((n_samples * self.sampling_rate, n_features))
        
      
        knn = NearestNeighbors(n_neighbors=self.k).fit(X)
        for i in range(len(X)):
            k_neighbors = knn.kneighbors(X[i].reshape(1,-1), 
                                         return_distance=False)[0]
            self.synthetic_samples(X, i, k_neighbors)
            
        if y is not None:
            return ( np.concatenate((self.synthetic, X, negative_X), axis=0), 
                     np.concatenate(([1]*(len(self.synthetic)+len(X)), y[y==0]), axis=0) )
        
        return np.concatenate((self.synthetic, X), axis=0)


    def synthetic_samples(self, X, i, k_neighbors):
        for j in range(self.sampling_rate):
            neighbor = np.random.choice(k_neighbors)
            diff = X[neighbor] - X[i]
            # creat new dataset
            self.synthetic[self.newindex] = X[i] + random.random() * diff
            self.newindex += 1